In [1]:
from google.colab import drive
import os
try:
    drive.flush_and_unmount()
except ValueError:
    pass

if os.path.exists('/content/drive') and os.path.isdir('/content/drive'):
    if os.listdir('/content/drive'):
        print("Mount point not empty, attempting removal.")
        # This might require manual user intervention or a shell command depending on permissions
        # For robustness, using a shell command to force removal might be necessary
        !rm -rf /content/drive/*
        !rmdir /content/drive || true

drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
!pip install --no-cache-dir \
    pymupdf pdfplumber tabula-py faiss-cpu rank-bm25 unidecode rapidfuzz \
    pydantic==2.7.1 transformers==4.30.0 \
    sentence-transformers==2.2.0 \
    huggingface-hub==0.23.0

!pip install huggingface-hub==0.25.2
!pip install -U sentence-transformers
!pip install gradio
!pip install gtts
!pip install speechrecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 255.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 199.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 133.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 147.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 241.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 194.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 189.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 192.3 MB/s eta 0:00:0

In [3]:
#Imports and basic config
import os, json, uuid, re, math, time, gc, glob
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional, Tuple

import fitz
import pdfplumber
import pandas as pd
from unidecode import unidecode
from rapidfuzz import fuzz
from rank_bm25 import BM25Okapi

import numpy as np
import faiss

# Embeddings (small, multilingual)
from sentence_transformers import SentenceTransformer

import tabula

In [ ]:
# USER INPUTS
# 1) Mount Google Drive manually in Colab (Runtime > Connect to Drive) or upload PDFs to /content/pdfs
PDF_DIR = "/content/drive/MyDrive/AgriBot_Project_3/content/pdfs"  # change to your folder path if different
INDEX_DIR = "/content/drive/MyDrive/AgriBot_Project_3/content/index"  # where we will save FAISS and metadata
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(INDEX_DIR, exist_ok=True)

# 2) API keys (store in environment for later steps)
os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"
os.environ["OPENWEATHER_API_KEY"] = "YOUR_API_KEY"
os.environ["TAVILY_API_KEY"] = "YOUR_API_KEY"
os.environ["AGROMONITORING_API_KEY"] = "YOUR_API_KEY"
os.environ["DATA_GOV_IN_API_KEY"] = "YOUR_API_KEY"

# Chunking/Embedding constants
TEXT_CHUNK_TOKENS = 450
TEXT_CHUNK_OVERLAP = 64
ROW_KEY_DIMENSIONS = ["crop","growth_stage","soil_type","region"]
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # small, multilingual

# Retrieval configuration
RETRIEVE_K_DENSE = 8
RETRIEVE_K_SPARSE = 8
MERGE_K = 5  # cap after merge
RERANK_TOP = 3 # we will implement remote reranker later with Groq

In [5]:

#Defining schema dataclasses
@dataclass
class QualityFlags:
    extracted_ok: bool
    table_parse_conf: float
    staleness_days: Optional[int] = None

@dataclass
class Citations:
    page_no: Optional[int] = None
    table_id: Optional[str] = None
    row_key: Optional[str] = None

@dataclass
class ChunkMetadata:
    doc_id: str
    title: str
    publisher: Optional[str]
    date: Optional[str]  # "YYYY-MM-DD" if known
    doc_type: Optional[str]
    domains: List[str]
    crops: List[str]
    regions: List[str]
    growth_stage: Optional[str]
    language: str  # "en" | "hi" | mixed
    page_no: Optional[int]
    section_heading: Optional[str]
    table_id: Optional[str]
    table_caption: Optional[str]
    units_system: str
    quality_flags: QualityFlags
    citations: Citations

@dataclass
class Chunk:
    id: str
    chunk_type: str  # "text" | "table_schema" | "table_row"
    text: str
    bm25_text: str
    metadata: ChunkMetadata
    # embeddings vector stored separately

def norm_text(s: str) -> str:
    # Normalize whitespace and convert smart quotes, remove excessive spaces
    s = s.replace("\xa0"," ").strip()
    s = re.sub(r"\s+", " ", s)
    return s

def detect_language(text: str) -> str:
    # Simple heuristic: presence of Devanagari unicode range → hi; else en/mixed
    if re.search(r"[\u0900-\u097F]", text):
        return "hi"
    return "en"

# Basic synonym maps
CROP_SYNONYMS = {
    "chana": ["bengal gram","gram","चना"],
    "paddy": ["rice","धान","चावल"],
    "wheat": ["गेहूं","gehu","gehun"],
    "maize": ["corn","मक्का","bhutta","makka"],
    "mustard": ["sarson","सरसों"],
    "okra": ["bhindi","भिंडी"],
    "cotton": ["कपास","kapas"]
}

REGION_SYNONYMS = {
    "uttar pradesh": ["up","उत्तर प्रदेश"],
    "madhya pradesh": ["mp","मध्य प्रदेश"],
    "bihar": ["बिहार"],
    "maharashtra": ["mh","महाराष्ट्र"],
    "tamil nadu": ["tn","तमिलनाडु"]
}

def normalize_units(cell: str) -> str:
    # Simple pass-through; expand to convert e.g., l/ha, kg/ha normalization
    if cell is None:
        return ""
    return norm_text(cell)

def build_row_key(row: Dict[str, Any]) -> str:
    parts = []
    for k in ROW_KEY_DIMENSIONS:
        v = row.get(k, "")
        parts.append(f"{k}={str(v).lower()}")
    return "|".join(parts)

def sanitize_table_df(df: pd.DataFrame) -> pd.DataFrame:
    # Ensure string col names
    df.columns = [str(c).strip() if c is not None else "" for c in df.columns]
    # make unique names if repeated
    seen = {}
    new_cols = []
    for c in df.columns:
        if c not in seen:
            seen[c] = 0
            new_cols.append(c)
        else:
            seen[c] += 1
            new_cols.append(f"{c}_{seen[c]}")
    df.columns = new_cols
    # Drop columns that are entirely empty or NaN
    df = df.dropna(axis=1, how="all")
    # Drop fully empty rows
    df = df.dropna(axis=0, how="all")
    # Fill remaining NaN with empty strings for robust string ops
    df = df.fillna("")
    # Convert everything to str for consistent downstream parsing
    for c in df.columns:
        df[c] = df[c].astype(str).map(lambda x: x.strip())
    return df



In [6]:
#Extract text chunks from PDF with section/page awareness
def chunk_text(text: str, max_tokens=TEXT_CHUNK_TOKENS, overlap=TEXT_CHUNK_OVERLAP) -> List[str]:
    # Token proxy by words
    words = text.split()
    chunks = []
    i = 0
    step = max_tokens - overlap
    while i < len(words):
        chunk_words = words[i:i+max_tokens]
        if not chunk_words:
            break
        chunks.append(" ".join(chunk_words))
        i += step
    return chunks

def extract_text_chunks(pdf_path: str, doc_meta_base: Dict[str, Any]) -> List[Chunk]:
    doc = fitz.open(pdf_path)
    chunks = []
    title_guess = os.path.basename(pdf_path)
    for page_no in range(len(doc)):
        page = doc.load_page(page_no)
        text = page.get_text("text")
        text = norm_text(text)
        if not text:
            continue
        lang = detect_language(text)
        # naive heading detection: first line or lines in caps
        heading = None
        lines = text.split("\n")
        if lines:
            cand = lines[0].strip()
            if len(cand) > 0 and (cand.isupper() or len(cand) < 80):
                heading = cand
        # chunk
        for piece in chunk_text(text):
            chunk_id = str(uuid.uuid4())
            md = ChunkMetadata(
                doc_id=doc_meta_base["doc_id"],
                title=doc_meta_base.get("title", title_guess),
                publisher=doc_meta_base.get("publisher"),
                date=doc_meta_base.get("date"),
                doc_type=doc_meta_base.get("doc_type", "guideline"),
                domains=doc_meta_base.get("domains", []),
                crops=doc_meta_base.get("crops", []),
                regions=doc_meta_base.get("regions", []),
                growth_stage=None,
                language=lang,
                page_no=page_no+1,
                section_heading=heading,
                table_id=None,
                table_caption=None,
                units_system="metric",
                quality_flags=QualityFlags(True, 1.0, None),
                citations=Citations(page_no=page_no+1, table_id=None, row_key=None),
            )
            chunks.append(Chunk(
                id=chunk_id,
                chunk_type="text",
                text=piece,
                bm25_text=piece,
                metadata=md
            ))
    doc.close()
    return chunks


In [8]:
#Extract tables with pdfplumber/tabula
def extract_tables(pdf_path: str, doc_meta_base: Dict[str, Any]) -> List[Chunk]:
    chunks = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            try:
                tables = page.extract_tables()
            except Exception:
                tables = []
            for t_idx, table in enumerate(tables or []):
                df = pd.DataFrame(table[1:], columns=table[0])
                df = df.dropna(how="all")
                table_id = f"{os.path.basename(pdf_path)}_p{i+1}_t{t_idx+1}"
                cols = []
                for c in df.columns:
                    cols.append({"name": str(c), "unit": None, "description": None})
                schema_text = "Table columns: " + ", ".join([c["name"] for c in cols])
                chunk_id = str(uuid.uuid4())
                md = ChunkMetadata(
                    doc_id=doc_meta_base["doc_id"],
                    title=doc_meta_base.get("title", os.path.basename(pdf_path)),
                    publisher=doc_meta_base.get("publisher"),
                    date=doc_meta_base.get("date"),
                    doc_type=doc_meta_base.get("doc_type", "guideline"),
                    domains=doc_meta_base.get("domains", []),
                    crops=doc_meta_base.get("crops", []),
                    regions=doc_meta_base.get("regions", []),
                    growth_stage=None,
                    language="en",
                    page_no=i+1,
                    section_heading=None,
                    table_id=table_id,
                    table_caption=None,
                    units_system="metric",
                    quality_flags=QualityFlags(True, 0.9, None),
                    citations=Citations(page_no=i+1, table_id=table_id, row_key=None),
                )
                chunks.append(Chunk(
                    id=chunk_id,
                    chunk_type="table_schema",
                    text=schema_text,
                    bm25_text=schema_text,
                    metadata=md
                ))
                # row chunks
                for _, row in df.iterrows():
                    row_dict = {}
                    for c in df.columns:
                        row_dict[str(c).strip().lower()] = normalize_units(str(row[c]) if pd.notna(row[c]) else "")
                    row_key = build_row_key({
                        "crop": row_dict.get("crop",""),
                        "growth_stage": row_dict.get("stage",""),
                        "soil_type": row_dict.get("soil",""),
                        "region": row_dict.get("region","")
                    })
                    # render compact textual row
                    parts = [f"{k}: {v}" for k,v in row_dict.items()]
                    row_text = f"Row ({row_key}) | " + " | ".join(parts)
                    chunk_id = str(uuid.uuid4())
                    md_row = md
                    md_row = ChunkMetadata(**asdict(md))
                    md_row.citations = Citations(page_no=i+1, table_id=table_id, row_key=row_key)
                    chunks.append(Chunk(
                        id=chunk_id,
                        chunk_type="table_row",
                        text=row_text,
                        bm25_text=row_text,
                        metadata=md_row
                    ))
    return chunks

def extract_tables(pdf_path: str, doc_meta_base: Dict[str, Any]) -> List[Chunk]:
    chunks = []
    base_title = doc_meta_base.get("title", os.path.basename(pdf_path))

    def add_table_chunks(df: pd.DataFrame, page_num: int, t_idx: int):
        nonlocal chunks
        if df is None or df.empty:
            return
        df = sanitize_table_df(df)
        if df.empty:
            return
        table_id = f"{os.path.basename(pdf_path)}_p{page_num}_t{t_idx}"
        # Schema chunk
        cols = [{"name": c, "unit": None, "description": None} for c in df.columns]
        schema_text = "Table columns: " + ", ".join([c["name"] for c in cols])
        md_schema = ChunkMetadata(
            doc_id=doc_meta_base["doc_id"],
            title=base_title,
            publisher=doc_meta_base.get("publisher"),
            date=doc_meta_base.get("date"),
            doc_type=doc_meta_base.get("doc_type", "guideline"),
            domains=doc_meta_base.get("domains", []),
            crops=doc_meta_base.get("crops", []),
            regions=doc_meta_base.get("regions", []),
            growth_stage=None,
            language="en",
            page_no=page_num,
            section_heading=None,
            table_id=table_id,
            table_caption=None,
            units_system="metric",
            quality_flags=QualityFlags(True, 0.9, None),
            citations=Citations(page_no=page_num, table_id=table_id, row_key=None),
        )
        chunks.append(Chunk(
            id=str(uuid.uuid4()),
            chunk_type="table_schema",
            text=schema_text,
            bm25_text=schema_text,
            metadata=md_schema
        ))
        # Row chunks
        for _, row in df.iterrows():
            row_dict = {str(c).strip().lower(): normalize_units(str(row[c])) for c in df.columns}
            row_key = build_row_key({
                "crop": row_dict.get("crop",""),
                "growth_stage": row_dict.get("stage",""),
                "soil_type": row_dict.get("soil",""),
                "region": row_dict.get("region","")
            })
            row_text = f"Row ({row_key}) | " + " | ".join([f"{k}: {v}" for k,v in row_dict.items()])
            md_row = ChunkMetadata(
                doc_id=md_schema.doc_id,
                title=md_schema.title,
                publisher=md_schema.publisher,
                date=md_schema.date,
                doc_type=md_schema.doc_type,
                domains=md_schema.domains,
                crops=md_schema.crops,
                regions=md_schema.regions,
                growth_stage=None,
                language=md_schema.language,
                page_no=page_num,
                section_heading=None,
                table_id=table_id,
                table_caption=None,
                units_system="metric",
                quality_flags=QualityFlags(True, 0.9, None),
                citations=Citations(page_no=page_num, table_id=table_id, row_key=row_key),
            )
            chunks.append(Chunk(
                id=str(uuid.uuid4()),
                chunk_type="table_row",
                text=row_text,
                bm25_text=row_text,
                metadata=md_row
            ))

    # Pass 1: pdfplumber
    pages_had_tables = False
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            try:
                tables = page.extract_tables()
            except Exception:
                tables = []
            if not tables:
                continue
            pages_had_tables = True
            for t_idx, table in enumerate(tables, start=1):
                try:
                    if not table or len(table) < 2:
                        continue
                    header = table[0]
                    rows = table[1:]
                    # Replace None or empty strings; generate synthetic headers if needed
                    if not header or all([(h is None or str(h).strip()=="") for h in header]):
                        ncols = max(len(r) for r in rows) if rows else 0
                        header = [f"col_{j+1}" for j in range(ncols)]
                    # Ensure row lengths match header
                    clean_rows = []
                    for r in rows:
                        if len(r) < len(header):
                            r = list(r) + [""]*(len(header)-len(r))
                        elif len(r) > len(header):
                            r = r[:len(header)]
                        clean_rows.append(r)
                    df = pd.DataFrame(clean_rows, columns=header)
                    df = sanitize_table_df(df)
                    if not df.empty:
                        add_table_chunks(df, page_num=i+1, t_idx=t_idx)
                except Exception as e:
                    pass # Continue without killing the whole PDF

    # Pass 2: Tabula fallback if no tables found via pdfplumber
    if not pages_had_tables:
        try:
            # Try lattice first (works for ruled tables)
            dfs = tabula.read_pdf(pdf_path, pages="all", lattice=True, multiple_tables=True)
            if not dfs:
                # Try stream for whitespace-separated tables
                dfs = tabula.read_pdf(pdf_path, pages="all", stream=True, multiple_tables=True)
            if dfs:
                for idx, df in enumerate(dfs, start=1):
                    try:
                        df = sanitize_table_df(df)
                        if not df.empty:
                            add_table_chunks(df, page_num=idx, t_idx=idx)
                    except Exception:
                        pass
        except Exception:
            pass

    return chunks

In [9]:


#Build hybrid index classes
class HybridIndex:
    def __init__(self, embedding_model_name=EMBEDDING_MODEL_NAME):
        self.model = SentenceTransformer(embedding_model_name)
        self.faiss_index = None
        self.embeddings = None
        self.bm25 = None
        self.corpus_texts = []
        self.chunks: List[Chunk] = []

    def add_chunks(self, new_chunks: List[Chunk]):
        self.chunks.extend(new_chunks)

    def build(self):
        # Prepare texts
        self.corpus_texts = [c.text for c in self.chunks]
        # Sparse
        tokenized = [t.split() for t in self.corpus_texts]
        self.bm25 = BM25Okapi(tokenized)
        # Dense
        emb = self.model.encode(self.corpus_texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
        self.embeddings = emb.astype("float32")
        d = self.embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(d)  # inner product for normalized vectors
        self.faiss_index.add(self.embeddings)

    def save(self, index_dir=INDEX_DIR):
        os.makedirs(index_dir, exist_ok=True)
        # Save FAISS
        faiss.write_index(self.faiss_index, os.path.join(index_dir, "faiss.index"))
        # Save embeddings
        np.save(os.path.join(index_dir, "embeddings.npy"), self.embeddings)
        # Save corpus + metadata
        with open(os.path.join(index_dir, "corpus_texts.json"), "w") as f:
            json.dump(self.corpus_texts, f)
        with open(os.path.join(index_dir, "chunks.jsonl"), "w") as f:
            for c in self.chunks:
                f.write(json.dumps({
                    "id": c.id,
                    "chunk_type": c.chunk_type,
                    "text": c.text,
                    "bm25_text": c.bm25_text,
                    "metadata": asdict(c.metadata)
                }) + "\n")

    def load(self, index_dir=INDEX_DIR):
        # Load FAISS
        self.faiss_index = faiss.read_index(os.path.join(index_dir, "faiss.index"))
        self.embeddings = np.load(os.path.join(index_dir, "embeddings.npy"))
        with open(os.path.join(index_dir, "corpus_texts.json"), "r") as f:
            self.corpus_texts = json.load(f)
        self.chunks = []
        with open(os.path.join(index_dir, "chunks.jsonl"), "r") as f:
            for line in f:
                obj = json.loads(line)
                md = obj["metadata"]
                chunk = Chunk(
                    id=obj["id"],
                    chunk_type=obj["chunk_type"],
                    text=obj["text"],
                    bm25_text=obj["bm25_text"],
                    metadata=ChunkMetadata(
                        doc_id=md["doc_id"],
                        title=md["title"],
                        publisher=md.get("publisher"),
                        date=md.get("date"),
                        doc_type=md.get("doc_type"),
                        domains=md.get("domains", []),
                        crops=md.get("crops", []),
                        regions=md.get("regions", []),
                        growth_stage=md.get("growth_stage"),
                        language=md.get("language","en"),
                        page_no=md.get("page_no"),
                        section_heading=md.get("section_heading"),
                        table_id=md.get("table_id"),
                        table_caption=md.get("table_caption"),
                        units_system=md.get("units_system","metric"),
                        quality_flags=QualityFlags(**md.get("quality_flags", {"extracted_ok":True,"table_parse_conf":1.0,"staleness_days":None})),
                        citations=Citations(**md.get("citations", {}))
                    )
                )
                self.chunks.append(chunk)
        # Rebuild BM25
        tokenized = [t.split() for t in self.corpus_texts]
        self.bm25 = BM25Okapi(tokenized)

    def search(self, query: str, k_dense=RETRIEVE_K_DENSE, k_sparse=RETRIEVE_K_SPARSE, merge_k=MERGE_K):
        # Sparse
        sparse_scores = self.bm25.get_scores(query.split())
        top_sparse_idx = np.argsort(sparse_scores)[::-1][:k_sparse]
        # Dense
        q_emb = self.model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
        D, I = self.faiss_index.search(q_emb, k_dense)
        top_dense_idx = I[0]
        # Merge by normalized score
        scores = {}
        # normalize sparse
        if len(top_sparse_idx) > 0:
            smax = sparse_scores[top_sparse_idx[0]] if sparse_scores[top_sparse_idx[0]] > 0 else 1.0
        else:
            smax = 1.0
        for idx in top_sparse_idx:
            scores[idx] = scores.get(idx, 0.0) + (sparse_scores[idx]/smax)
        # normalize dense
        for rank, idx in enumerate(top_dense_idx):
            sim = D[0][rank]
            scores[idx] = scores.get(idx, 0.0) + float(sim)
        # sort and cap
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:merge_k]
        results = [(self.chunks[i], s) for i,s in ranked]
        return results

In [10]:
#Create a simple manifest by scanning PDF_DIR
def default_doc_meta(pdf_path: str) -> Dict[str, Any]:
    base = os.path.basename(pdf_path)
    # crude guesses
    return {
        "doc_id": str(uuid.uuid4()),
        "title": os.path.splitext(base)[0],
        "publisher": None,
        "date": None,
        "doc_type": "guideline",
        "domains": ["agronomy","irrigation","pest"],  # broad for now
        "crops": [],  # leave empty; retrieval will be semantic
        "regions": [],
    }

pdf_files = sorted(glob.glob(os.path.join(PDF_DIR, "*.pdf")))
print("Found PDFs:", len(pdf_files))
for p in pdf_files[:5]:
    print(" -", p)

Found PDFs: 19
 - /content/drive/MyDrive/AgriBot_Project_3/content/pdfs/0be491ed-827d-4828-95cc-9600a7fe0462.pdf
 - /content/drive/MyDrive/AgriBot_Project_3/content/pdfs/0d6684ab-defc-4fc1-9814-0ead3f9ae97f.pdf
 - /content/drive/MyDrive/AgriBot_Project_3/content/pdfs/319d5c19-a3f4-43ee-bac0-ac7198247ef7.pdf
 - /content/drive/MyDrive/AgriBot_Project_3/content/pdfs/367385bd-b0a9-4595-8006-fdec4ff4d8fc.pdf
 - /content/drive/MyDrive/AgriBot_Project_3/content/pdfs/6647be65-d94b-4f1b-ac6e-c585d1d0f2fb.pdf


In [11]:
# Run ingestion and build index
# hybrid = HybridIndex(embedding_model_name=EMBEDDING_MODEL_NAME)

# total_chunks = 0
# for pdf_path in pdf_files:
#     meta = default_doc_meta(pdf_path)
#     try:
#         t_chunks = extract_text_chunks(pdf_path, meta)
#         tb_chunks = extract_tables(pdf_path, meta)
#         hybrid.add_chunks(t_chunks + tb_chunks)
#         total_chunks += len(t_chunks) + len(tb_chunks)
#         print(f"Ingested {os.path.basename(pdf_path)}: text={len(t_chunks)}, table_chunks={len(tb_chunks)}")
#     except Exception as e:
#         print("Failed:", pdf_path, e)

# print("Total chunks:", total_chunks)
# hybrid.build()
# hybrid.save(INDEX_DIR)
# print("Index built and saved to", INDEX_DIR)
# hybrid = HybridIndex(embedding_model_name=EMBEDDING_MODEL_NAME)

In [12]:
# Test search
hybrid2 = HybridIndex(embedding_model_name=EMBEDDING_MODEL_NAME)
hybrid2.load(INDEX_DIR)

def preview_hit(hit, score):
    c = hit
    print(f"[{c.chunk_type}] score={score:.3f} page={c.metadata.page_no} table_id={c.metadata.table_id}")
    print(c.text[:300], "...")
    print("Doc:", c.metadata.title)
    print("—")

queries = [
    "Irrigation schedule for wheat during flowering stage",
    "Bhindi ko kitna paani dena chahiye garmi me",
    "Pest threshold for bollworm in cotton",
]

for q in queries:
    print("Q:", q)
    results = hybrid2.search(q)
    for chunk, sc in results[:3]:
        preview_hit(chunk, sc)
    print("="*80)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Q: Irrigation schedule for wheat during flowering stage
[table_row] score=1.307 page=3 table_id=6647be65-d94b-4f1b-ac6e-c585d1d0f2fb.pdf_p3_t1
Row (crop=|growth_stage=|soil_type=|region=) | commodity report: wheat
key parameters on commodity situation are discussed below
crop calender:
wheat is sown in rabi season.
figure 1:crop calendar
source: icar
production:
as per da&fw third advance estimates for 2024-25 ,wheat production is estimate ...
Doc: 6647be65-d94b-4f1b-ac6e-c585d1d0f2fb
—
[table_row] score=1.000 page=65 table_id=farmerbook.pdf_p65_t2
Row (crop=|growth_stage=|soil_type=|region=) | s symptoms: per Pest starts attacking during flowering season. | control measures: Spraying of acetamiprid or thiamethoxam (0.2gm/litre of water). ...
Doc: farmerbook
—
[text] score=0.730 page=61 table_id=None
dipped in a solution of aurofugine1 (10 g / 100 litres of water) for 1.5 hour prior to planting. Tissue culture plant: Select healthy and uniform plants. Irrigation and scheduling: The wat

# **Phase 2**

In [13]:
from typing import Tuple
import time, hashlib, json
import requests
from datetime import datetime
import re


In [14]:
# Simple intent classifier
def classify_intent(query: str) -> str:
    q = query.lower()
    if any(w in q for w in ["irrigation", "paani", "water"]):
        return "irrigation"
    if any(w in q for w in ["pest", "keeda", "bollworm", "disease", "threshold", "etl"]):
        return "pest_disease"
    if any(w in q for w in ["price", "daam", "mandi", "rate", "bazaar"]):
        return "market_price"
    if any(w in q for w in ["policy", "yojana", "scheme", "kcc", "pm-kisan"]):
        return "policy"
    if any(w in q for w in ["weather", "mausam", "rain", "temperature", "baarish"]):
        return "weather"
    return "general"

# Query expansion based on intent
def expand_query(query: str, intent: str) -> str:
    q = query
    if intent == "irrigation":
        q += " irrigation schedule water requirement mm per irrigation frequency critical stages guideline"
    elif intent == "pest_disease":
        q += " economic threshold level ETL control measures official agri advisory"
    elif intent == "market_price":
        q += " mandi daily price market rates arrivals agmarknet"
    return q.strip()

def retrieve_with_confidence(query: str, hybrid_idx: HybridIndex,
                              k_dense=RETRIEVE_K_DENSE, k_sparse=RETRIEVE_K_SPARSE) -> Tuple[list, float, str]:
    # Classify and expand query
    intent = classify_intent(query)
    expanded_q = expand_query(query, intent)

    # Search hybrid index
    results = hybrid_idx.search(expanded_q, k_dense=k_dense, k_sparse=k_sparse)

    boosted_results = []
    for chunk, score in results:
        boost = 0.0
        # Prefer table rows for agronomy/pest
        if chunk.chunk_type == "table_row":
            boost += 0.4
        # Boost based on domain match
        if intent == "irrigation" and "irrigation" in (chunk.metadata.domains or []):
            boost += 0.3
        if intent == "pest_disease" and any(d in chunk.metadata.domains for d in ["pest","disease"]):
            boost += 0.3
        # Down-weight irrelevant docs
        if any(bad in (chunk.metadata.title or "").lower()
               for bad in ["insurance","pmfby","market intelligence","scheme","policy"]):
            boost -= 0.5

        boosted_results.append((chunk, score + boost))

    boosted_results = sorted(boosted_results, key=lambda x: x[1], reverse=True)

    # Confidence = top score normalized against arbitrary scale
    top_score = boosted_results[0][1] if boosted_results else 0
    confidence = min(1.0, max(0.0, top_score / 2.0))  # scale to 0-1

    return boosted_results[:RERANK_TOP], confidence, intent



In [15]:
test_queries = [
    "Irrigation schedule for wheat during flowering stage",
    "Bhindi ko kitna paani dena chahiye garmi me",
    "Pest threshold for bollworm in cotton"
]

for q in test_queries:
    hits, conf, intent = retrieve_with_confidence(q, hybrid2)
    print("="*80)
    print(f"Q: {q} | Intent: {intent} | Confidence: {conf:.2f}")
    for chunk, sc in hits:
        print(f"[{chunk.chunk_type}] boosted_score={sc:.3f} page={chunk.metadata.page_no}")
        print("Doc:", chunk.metadata.title)
        print("Text:", chunk.text[:200], "...")

Q: Irrigation schedule for wheat during flowering stage | Intent: irrigation | Confidence: 0.88
[text] boosted_score=1.759 page=26
Doc: farmerbook
Text: 18 General Conditions for Cultivation of Crops Farmer’s Handbook on Basic Agriculture Crop Yield increase (%) Water saving (%) Sugarcane 133.3 49.3 Cotton 88.0 46.6 Onion 53.8 46.1 Potato 79.5 54.1 Be ...
[table_row] boosted_score=1.700 page=4
Doc: Guidelines_PMKSY
Text: Row (crop=|growth_stage=|soil_type=|region=) | extension activities for promotion of scientific moisture conservation and: available water including rainfall and minimise irrigation requirement (Jal ...
[text] boosted_score=1.284 page=61
Doc: farmerbook
Text: dipped in a solution of aurofugine1 (10 g / 100 litres of water) for 1.5 hour prior to planting. Tissue culture plant: Select healthy and uniform plants. Irrigation and scheduling: The water requireme ...
Q: Bhindi ko kitna paani dena chahiye garmi me | Intent: irrigation | Confidence: 0.85
[table_row] boosted_sco

In [16]:
CACHE = {}
CACHE_TTL = {
    "weather": 600,       # 10 minutes
    "soil": 10800,        # 3 hours
    "ndvi": 86400,        # 24 hours
    "market": 86400,      # 24 hours
    "web": 21600          # 6 hours
}

def make_cache_key(name: str, params: dict) -> str:
    raw = name + json.dumps(params, sort_keys=True)
    return hashlib.md5(raw.encode()).hexdigest()

def cache_get(name, params):
    key = make_cache_key(name, params)
    if key in CACHE:
        item = CACHE[key]
        if time.time() - item["time"] < CACHE_TTL.get(name, 3600):
            return item["data"]
    return None

def cache_set(name, params, data):
    key = make_cache_key(name, params)
    CACHE[key] = {"data": data, "time": time.time()}

def get_weather(lat, lon):
    params = {"lat": lat, "lon": lon, "appid": os.environ["OPENWEATHER_API_KEY"], "units": "metric"}
    cached = cache_get("weather", params)
    if cached:
        return cached

    url = "https://api.openweathermap.org/data/2.5/onecall"
    try:
        r = requests.get(url, params=params, timeout=5)
        r.raise_for_status()
        data = r.json()
        cache_set("weather", params, data)
        return data
    except Exception as e:
        return {"error": str(e)}

def get_soil_and_ndvi(polygon_id):
    soil_params = {"polyid": polygon_id, "appid": os.environ["AGROMONITORING_API_KEY"]}
    cached_soil = cache_get("soil", soil_params)
    if not cached_soil:
        try:
            soil_url = f"http://api.agromonitoring.com/agro/1.0/soil"
            r = requests.get(soil_url, params=soil_params, timeout=5)
            r.raise_for_status()
            cached_soil = r.json()
            cache_set("soil", soil_params, cached_soil)
        except Exception as e:
            cached_soil = {"error": str(e)}

    ndvi_params = {"polyid": polygon_id, "appid": os.environ["AGROMONITORING_API_KEY"]}
    cached_ndvi = cache_get("ndvi", ndvi_params)
    if not cached_ndvi:
        try:
            ndvi_url = f"http://api.agromonitoring.com/agro/1.0/ndvi/history"
            r = requests.get(ndvi_url, params=ndvi_params, timeout=5)
            r.raise_for_status()
            cached_ndvi = r.json()
            cache_set("ndvi", ndvi_params, cached_ndvi)
        except Exception as e:
            cached_ndvi = {"error": str(e)}

    return {"soil": cached_soil, "ndvi": cached_ndvi}

# helper to extract state/district/market from query
STATE_LIST = [
    "Uttar Pradesh", "Madhya Pradesh", "West Bengal", "Bihar", "Maharashtra",
    "Punjab", "Haryana", "Rajasthan", "Gujarat", "Karnataka", "Andhra Pradesh",
    "Tamil Nadu", "Odisha", "Telangana", "Chhattisgarh", "Jharkhand"
]

def extract_location_from_query(query):
    found_state = None
    for s in STATE_LIST:
        if s.lower() in query.lower():
            found_state = s
            break
    return found_state  # we can extend later for district/market

def get_market_price(commodity, state=None, district=None, market=None):
    resource_id = "35985678-0d79-46b4-9ed6-6f13308a1d24"
    params = {
        "api-key": os.environ["DATA_GOV_IN_API_KEY"],
        "format": "json",
        "limit": 3,
        "filters[commodity]": commodity
    }
    if state:
        params["filters[state]"] = state
    if district:
        params["filters[district]"] = district
    if market:
        params["filters[market]"] = market

    cached = cache_get("market", params)
    if cached:
        return cached

    try:
        url = f"https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24"
        r = requests.get(url, params=params, timeout=3)
        r.raise_for_status()
        data = r.json()
        cache_set("market", params, data)
        return data
    except Exception as e:
        return {"error": str(e)}

result = get_market_price(
    commodity="Wheat",
    state="Uttar Pradesh"
)

for rec in result.get("records", []):
    print(f"{rec['market']} ({rec['district']}): Modal Price ₹{rec['modal_price']} on {rec['arrival_date']}")

def tavily_search(query):
    params = {
        "query": query + " site:gov.in OR site:nic.in OR site:imd.gov.in OR site:agmarknet.gov.in OR site:enam.gov.in",
        "api_key": os.environ["TAVILY_API_KEY"]
    }
    cached = cache_get("web", params)
    if cached:
        return cached

    url = "https://api.tavily.com/search"
    try:
        r = requests.post(url, json=params, timeout=6)
        r.raise_for_status()
        data = r.json()
        if isinstance(data, dict) and "results" in data and isinstance(data["results"], list):
            data["results"] = data["results"][:3]
        cache_set("web", params, data)
        return data
    except Exception as e:
        return {"error": str(e)}


def router(query):
    hits, conf, intent = retrieve_with_confidence(query, hybrid2)

    # High confidence KB answer
    if conf >= 1.99 and intent not in ["market_price", "weather"]:
        return {
            "source": "KB",
            "answer": hits[0][0].text,
            "confidence": conf
        }

    # Market price handler: Call Agmarknet live API
    if intent == "market_price":
        words = query.lower().split()
        commodity = None
        state = None
        for w in words:
            if w in ["wheat", "chana", "rice", "paddy", "mustard", "cotton", "okra"]:
                commodity = w.capitalize()
        if not commodity:
            commodity = "Wheat"

        price_data = get_market_price(commodity, state=state)

        if "records" in price_data and price_data["records"]:
            lines = []
            for rec in price_data["records"]:
                lines.append(f"{rec['market']} ({rec['district']}): ₹{rec['modal_price']} on {rec['arrival_date']}")
            return {
                "source": "Market API (Agmarknet)",
                "answer": "\n".join(lines),
                "confidence": conf
            }
        else:
            return {
                "source": "Market API (Agmarknet)",
                "answer": "No recent price data available for your query.",
                "confidence": conf
            }

    # Weather handler
    if intent == "weather":
        weather_data = get_weather(28.6139, 77.2090)
        return {"source": "Weather API", "answer": weather_data, "confidence": conf}

    # For irrigation/pest questions:
    if intent in ["pest_disease", "irrigation"]:
        if conf >= 1.99:
            return {"source": "KB", "answer": hits[0][0].text, "confidence": conf}
        else:
            web_data = tavily_search(query)
            return {"source": "Web (Tavily)", "answer": web_data, "confidence": conf}

    # Fallback for general queries
    web_data = tavily_search(query)
    return {"source": "Web (Tavily)", "answer": web_data, "confidence": conf}



In [17]:
queries = [
    "Current mandi price for wheat in Uttar Pradesh",
    "Mandi price of chana in Madhya Pradesh",
    "Modal rate of paddy in West Bengal",
    "How is weather in kolkata"
]

for q in queries:
    print("="*80)
    res = router(q)
    print(f"Q: {q}")
    print("Source:", res["source"])
    print("Confidence:", res["confidence"])
    print("Answer:\n", res["answer"])

Q: Current mandi price for wheat in Uttar Pradesh
Source: Market API (Agmarknet)
Confidence: 1.0
Answer:
 No recent price data available for your query.
Q: Mandi price of chana in Madhya Pradesh
Source: Market API (Agmarknet)
Confidence: 0.7
Answer:
 No recent price data available for your query.
Q: Modal rate of paddy in West Bengal
Source: Market API (Agmarknet)
Confidence: 0.6778044537852479
Answer:
 No recent price data available for your query.
Q: How is weather in kolkata
Source: Weather API
Confidence: 0.5
Answer:
 {'error': '401 Client Error: Unauthorized for url: https://api.openweathermap.org/data/2.5/onecall?lat=28.6139&lon=77.209&appid=878e6ec90aca2d5c58dc76f880b407cd&units=metric'}


In [18]:
def get_weather_simple(city_name):
    params = {"q": city_name, "appid": os.environ["OPENWEATHER_API_KEY"], "units": "metric"}
    url_current = "https://api.openweathermap.org/data/2.5/weather"
    try:
        r = requests.get(url_current, params=params, timeout=5)
        r.raise_for_status()
        data = r.json()
        return {
            "temp": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind": data["wind"]["speed"]
        }
    except Exception as e:
        return {"error": str(e)}

def router(query):
    hits, conf, intent = retrieve_with_confidence(query, hybrid2)

    # High confidence KB
    if conf >= 0.99 and intent not in ["market_price", "weather"]:
        return {
            "source": "KB",
            "answer": hits[0][0].text,
            "confidence": conf
        }

    # MARKET PRICE
    if intent == "market_price":
        state = extract_location_from_query(query)
        commodity = None
        for w in query.lower().split():
            if w in ["wheat", "chana", "rice", "paddy", "mustard", "cotton", "okra"]:
                commodity = w.capitalize()
        if not commodity:
            commodity = "Wheat"

        price_data = get_market_price(commodity, state=state)

        if "records" in price_data and price_data["records"]:
            header = f"Commodity: {commodity} | State: {state or 'All India'}"
            lines = [header]
            for rec in price_data["records"]:
                lines.append(f"{rec['market']} ({rec['district']}): ₹{rec['modal_price']} on {rec['arrival_date']}")
            return {
                "source": "Market API (Agmarknet)",
                "answer": "\n".join(lines),
                "confidence": conf
            }
        else:
            return {
                "source": "Market API (Agmarknet)",
                "answer": "No recent price data available for your query.",
                "confidence": conf
            }

    # WEATHER
    if intent == "weather":
        # TEMPORARY: use city simple API
        city = query.split()[-1]  # naive parse
        weather_data = get_weather_simple(city)
        return {"source": "Weather API", "answer": weather_data, "confidence": conf}

    # PEST/IRRIGATION
    if intent in ["pest_disease", "irrigation"]:
        if conf >= 0.5:
            return {"source": "KB", "answer": hits[0][0].text, "confidence": conf}
        else:
            web_data = tavily_search(query)
            return {"source": "Web (Tavily)", "answer": web_data, "confidence": conf}

    # Fallback
    web_data = tavily_search(query)
    return {"source": "Web (Tavily)", "answer": web_data, "confidence": conf}



In [19]:
queries = [
    "Current mandi price for wheat in Uttar Pradesh",
    "Mandi price of chana in Madhya Pradesh",
    "Modal rate of paddy in West Bengal",
    "How is weather in kolkata"
]

for q in queries:
    print("="*80)
    res = router(q)
    print(f"Q: {q}")
    print("Source:", res["source"])
    print("Confidence:", res["confidence"])
    print("Answer:\n", res["answer"])

Q: Current mandi price for wheat in Uttar Pradesh
Source: Market API (Agmarknet)
Confidence: 1.0
Answer:
 No recent price data available for your query.
Q: Mandi price of chana in Madhya Pradesh
Source: Market API (Agmarknet)
Confidence: 0.7
Answer:
 No recent price data available for your query.
Q: Modal rate of paddy in West Bengal
Source: Market API (Agmarknet)
Confidence: 0.6778044537852479
Answer:
 No recent price data available for your query.
Q: How is weather in kolkata
Source: Weather API
Confidence: 0.5
Answer:
 {'temp': 27.97, 'description': 'haze', 'humidity': 94, 'wind': 4.12}


# **Phase 3**

In [ ]:
!pip install groq

from groq import Groq
import os

# Set your key
os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 5.7 MB/s eta 0:00:00


In [21]:
tools_def = [
    {
        "type": "function",
        "function": {
            "name": "get_market_price",
            "description": "Fetch mandi price (Agmarknet) for a commodity",
            "parameters": {
                "type": "object",
                "properties": {
                    "commodity": {"type": "string", "description": "e.g., Wheat, Chana, Paddy"},
                    "state": {"type": "string"},
                    "district": {"type": "string"},
                    "market": {"type": "string"}
                },
                "required": ["commodity"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current/forecast weather; pass a city name or 'lat,lon'",
            "parameters": {
                "type": "object",
                "properties": {
                    "city_or_latlon": {"type": "string", "description": "e.g., Kolkata or '22.57,88.36'"}
                },
                "required": ["city_or_latlon"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Search official government sources for policies or advisories",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            }
        }
    }
]

def groq_stream_final_answer(messages, model="openai/gpt-oss-120b", temperature=0.2):
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=True
    )
    final_text = []
    for chunk in stream:
        delta = getattr(chunk.choices[0].delta, "content", None)
        if delta:
            print(delta, end="", flush=True)
            final_text.append(delta)
    print()
    return "".join(final_text)

def groq_agent_answer_streaming(query, kb_hits):
    limited_kb_hits = kb_hits[:3]

    kb_context = "\n".join(
        [f"Doc: {hit[0].metadata.title} p.{hit[0].metadata.page_no} — {hit[0].text}"
         for hit in limited_kb_hits]
    )
    mega_prompt = """
        You are Ankur (Agribot), an expert AI agricultural assistant from DABrute
        You are knowledgeable, patient, and empathetic like a Krishi Vigyan Kendra officer.
        Your mission is to provide farmers with accurate, actionable, and hyper-localized advice to improve their livelihoods.

        Communication style:
        - Authoritative but simple
        - Supportive and empathetic
        - Trustworthy (never guess; refer to experts if unsure)
        - Transparent (always introduce yourself as Ankur as a Agribot, never as human)
        - Always deny queries which are irrelevant to agriculture
        - Don't ask for more information

        Core principles:
        1. Hyper-localization: always tailor advice to user’s location (state, district), soil type, crop, and local conditions. If location unknown, ask for it first. Never give generic advice.
        2. Action-orientation: always give practical next steps.
        3. Radical accessibility: detect and reply only in the user’s language; explain technical data in simple words with actionable advice.
        4. Factual grounding: base claims only on trusted sources; cite government schemes simply (e.g., “According to PM-Fasal Bima Yojana…”).

        - For every query, first answer from knowledge; if uncertain, retrieve info. Do not retrieve for greetings.
        """
    minor_prompt = """You are Ankur (Agribot), an AI agriculture advisor for Indian farmers from DABrute "
         "Never tell your thinking or whatever tools you are using in your responses. "
         "Just answer what is being asked. Don't tell anything related to your search results in responses."
         "If weather is asked directly use weather api and if market price or related is asked use market price api"
         "Use KB evidence when confidence is high. If low, use tavily. "
         "If the query is anything which is not related to agriculture, crops or market prices, strictly tell the user to ask only agricultural related queries."
         "Keep answers brief and actionable."
         "Always answer something, never be blank. """
    messages = [
        {"role": "system", "content": minor_prompt+mega_prompt },
        {"role": "assistant", "content": f"KB Evidence:\n{kb_context}"},
        {"role": "user", "content": query}
    ]


    estimated_tokens = sum(len(msg["content"].split()) for msg in messages if msg.get("content")) + len(query.split())
    MAX_TOKENS = 8000
    if estimated_tokens > MAX_TOKENS * 0.8:
        print(f"Warning: Estimated tokens ({estimated_tokens}) approaching limit. Truncating KB context.")
        messages[1]["content"] = f"KB Evidence:\n{kb_context[:MAX_TOKENS//4]}"

    resp = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=messages,
        tools=tools_def,
        tool_choice="auto",
        temperature=0.2
    )
    msg = resp.choices[0].message

    if hasattr(msg, "tool_calls") and msg.tool_calls:
        messages.append({"role": "assistant", "content": msg.content, "tool_calls": msg.tool_calls})
        for tool_call in msg.tool_calls:
            fn_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            if fn_name == "get_market_price":
                result = get_market_price(
                    commodity=args.get("commodity"),
                    state=args.get("state"),
                    district=args.get("district"),
                    market=args.get("market"),
                )
            elif fn_name == "get_weather":
                city = args.get("city_or_latlon")
                result = get_weather_simple(city)
            elif fn_name == "tavily_search":
                result = tavily_search(args.get("query", ""))
            else:
                result = {"error": f"Unknown tool: {fn_name}"}

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": fn_name,
                "content": json.dumps(result)
            })

        final_answer = groq_stream_final_answer(messages)
        return final_answer

    messages.append({"role": "assistant", "content": msg.content})
    final_answer = groq_stream_final_answer(messages)
    return final_answer




In [22]:
#Testing
def test_groq_agent():
    tests = [
        # "Irrigation schedule for wheat during flowering stage",
        # "Watering tips for pea plants",
        # "Current mandi price for wheat in Uttar Pradesh",
        # "Weather forecast for Kolkata tomorrow",
        # "Latest crop insurance policy for PMFBY"
        "who is president of india?",
        # "Which crops should i sow in winter?",
        "Based on current weather in nagpur, what should be the best crop for profit?"
    ]
    for q in tests:
        print("="*100)
        hits, conf, intent = retrieve_with_confidence(q, hybrid2)
        print(f"User Query: {q}\n[DEBUG] Intent: {intent} | Confidence: {conf:.2f}\n")
        ans = groq_agent_answer_streaming(q, hits)
        # print("\n— end of answer —\n")

test_groq_agent()

User Query: who is president of india?
[DEBUG] Intent: general | Confidence: 0.70

I’m Ankur, an agricultural assistant. I can help with farming, crop, weather, or market‑price questions. Please ask a query related to agriculture.
User Query: Based on current weather in nagpur, what should be the best crop for profit?
[DEBUG] Intent: weather | Confidence: 0.25

**Ankur (Agribot) here – your Krishi Vigyan Kendra assistant.**  

**Current Nagpur weather (18 Aug 2025):** 26 °C, overcast, high humidity (≈94 %). This is typical mid‑monsoon conditions, ideal for Kharif crops that need good moisture.

### Most profitable Kharif crop for Nagpur now
**Soybean (Varieties: MAUS-2, JS‑335, or locally adapted hybrids).**  

- **Why it fits the weather:**  
  - Requires 500–750 mm rainfall; the monsoon is delivering that.  
  - Tolerates 25‑30 °C temperatures and high humidity.  
  - Harvestable in 90–110 days, allowing you to finish before the post‑monsoon heat.

- **Market advantage:**  
  - Soybe

In [23]:
def test_groq_agent():
    tests = [
        "price of bhindi in kharagpur"
    ]
    for q in tests:
        print("="*100)
        hits, conf, intent = retrieve_with_confidence(q, hybrid2)
        print(f"User Query: {q}\n[DEBUG] Intent: {intent} | Confidence: {conf:.2f}\n")
        ans = groq_agent_answer_streaming(q, hits)

        # print("\n— end of answer —\n")

test_groq_agent()

User Query: price of bhindi in kharagpur
[DEBUG] Intent: market_price | Confidence: 0.70

I’m Ankur, your Agribot. The market‑price database does not have a current entry for Bhindi in Kharagpur (Paschim Medinipur, West Bengal).  

**Next steps**
1. Visit the nearest APMC market (e.g., Kharagpur or Medinipur) tomorrow to note the arrival‑price board.  
2. Record the minimum, maximum and modal prices you see and share them with your local Krishi Vigyan Kendra for trend tracking.  
3. You can also check the Agmarknet website or the e-NAM portal later today for updates.  

If you need advice on cultivation practices, pest management, or government schemes for bhindi, feel free to ask!


# **Phase 4 : User Interfacce**

In [24]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import tempfile
from typing import List, Tuple
import os
import time

In [31]:
class AgriBot:
    """
    Encapsulates the logic for the AgriBot, integrating the backend functions
    from the notebook with the Gradio UI.
    """
    def __init__(self):
        self.recognizer = sr.Recognizer()

    def process_text_query(self, message: str, history: List[Tuple[str, str]]) -> Tuple[List[Tuple[str, str]], str]:
        """
        Processes a text query by retrieving context from the knowledge base (KB)
        and generating a response using the Groq agent.
        """
        history = history or []

        try:
            hits, confidence, intent = retrieve_with_confidence(message, hybrid2)

            history.append((message, None))

            response = groq_agent_answer_streaming(message, hits)

            history[-1] = (message, response)

            return history, ""
        except Exception as e:
            error_msg = f"An error occurred: {str(e)}"
            history.append((message, error_msg))
            return history, ""

    def text_to_speech(self, text: str) -> str:
        """Converts text to an MP3 audio file using gTTS."""
        if not text:
            return None
        try:
            tts = gTTS(text=text, lang='en', slow=False)
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as temp_fp:
                tts.save(temp_fp.name)
                return temp_fp.name
        except Exception as e:
            print(f"TTS Error: {e}")
            return None

    def speech_to_text(self, audio_filepath) -> str:
        """Converts an audio file to text using Google's Speech Recognition."""
        if audio_filepath is None:
            return "No audio recorded. Please try again."
        try:
            with sr.AudioFile(audio_filepath) as source:
                audio_data = self.recognizer.record(source)
                # Recognize speech using Google Web Speech API
                text = self.recognizer.recognize_google(audio_data)
                return text
        except sr.UnknownValueError:
            return "Could not understand the audio. Please speak clearly."
        except sr.RequestError as e:
            return f"Speech recognition service unavailable; {e}"
        except Exception as e:
            return f"An error occurred while processing audio: {e}"

# Initialize a single instance of the bot
agribot = AgriBot()

# **Testing**

#ANKUR UI


In [ ]:
import gradio as gr
from gtts import gTTS
import speech_recognition as sr
import os
import uuid
import time

green_theme = gr.themes.Base(
    primary_hue=gr.themes.colors.green,
    secondary_hue=gr.themes.colors.lime,
    neutral_hue=gr.themes.colors.slate,
    font=[gr.themes.GoogleFont("Inter"), "ui-sans-serif", "system-ui", "sans-serif"],
).set(
    button_primary_background_fill="*primary_500",
    button_primary_background_fill_hover="*primary_600",
)


def text_to_speech(text, lang='en'):
    """Converts text to an audio file."""
    if not text or not isinstance(text, str):
        print("Error: Invalid text provided for TTS.")
        return None
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        os.makedirs("/content/audio_outputs", exist_ok=True)
        audio_filename = f"/content/audio_outputs/{uuid.uuid4()}.mp3"
        tts.save(audio_filename)
        return audio_filename
    except Exception as e:
        print(f"Error during TTS conversion: {e}")
        return None

def speech_to_text(audio_filepath):
    """Transcribes speech from an audio file."""
    if not audio_filepath or not os.path.exists(audio_filepath):
        return ""

    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_filepath) as source:
            audio_data = recognizer.record(source)
        return recognizer.recognize_google(audio_data)
    except sr.UnknownValueError:
        return "Sorry, I could not understand the audio. Please try again."
    except sr.RequestError as e:
        return f"API Error: Could not request results; {e}"
    except Exception as e:
        print(f"An unexpected error in speech-to-text: {e}")
        return ""



chat_state = {
    "history": [],
    "last_response_audio": None
}


def ankur_chat_interface(user_text_input, user_audio_input):
    if user_audio_input:
        user_query = speech_to_text(user_audio_input)
    else:
        user_query = user_text_input
    if not user_query:
        return chat_state["history"], chat_state["last_response_audio"], ""


    hits, confidence, intent = retrieve_with_confidence(user_query, hybrid2)
    bot_response_text = groq_agent_answer_streaming(user_query, hits)

    bot_audio_response_path = text_to_speech(bot_response_text)
    chat_state["history"].append((user_query, bot_response_text))
    chat_state["last_response_audio"] = bot_audio_response_path

    return chat_state["history"], bot_audio_response_path, ""


def speak_last_response():
    return chat_state["last_response_audio"]



with gr.Blocks(theme=green_theme, css="footer {display: none !important}", title="Ankur The AgriBot") as demo:
    gr.Markdown("# Ankur The AgriBot 🌱")
    gr.Markdown("Ask me about weather updates, crop prices, or farming tips in real-time!")

    chatbot_display = gr.Chatbot(label="Chat with AgriBot", height=450, bubble_full_width=False)

    with gr.Row():
        text_input_box = gr.Textbox(placeholder="Type your farming question here...", show_label=False, container=False, scale=5)
        send_button = gr.Button("Send", variant="primary", scale=1, min_width=150)

    with gr.Accordion("🎤 Record Your Voice Question", open=False):
        voice_input_audio = gr.Audio(sources=["microphone"], type="filepath", label="Click the record button and speak your question clearly.")
        process_voice_button = gr.Button("Process Voice Input")

    with gr.Row():
        speak_last_button = gr.Button("🔊 Speak Last Response")
        response_audio_player = gr.Audio(label="Bot Response Audio", type="filepath", interactive=False, autoplay=False)


    text_submit_handler = send_button.click(fn=ankur_chat_interface, inputs=[text_input_box, gr.State(None)], outputs=[chatbot_display, response_audio_player, text_input_box])
    text_input_box.submit(fn=ankur_chat_interface, inputs=[text_input_box, gr.State(None)], outputs=[chatbot_display, response_audio_player, text_input_box])
    process_voice_button.click(fn=ankur_chat_interface, inputs=[gr.State(None), voice_input_audio], outputs=[chatbot_display, response_audio_player, text_input_box])
    speak_last_button.click(fn=speak_last_response, inputs=[], outputs=[response_audio_player])


demo.launch(debug=True, share=True)



/tmp/ipython-input-1235361134.py:88: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_display = gr.Chatbot(label="Chat with AgriBot", height=450, bubble_full_width=False)
/tmp/ipython-input-1235361134.py:88: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_display = gr.Chatbot(label="Chat with AgriBot", height=450, bubble_full_width=False)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://bf509defcd91403491.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Ankur (Agribot) – Soil best suited for tomatoes in Maharashtra**

- **Texture:** Loam → sandy‑loam (good drainage, enough water‑holding capacity).  
- **pH:** 6.0 – 6.8 (slightly acidic to neutral).  
- **Organic matter:** ≥ 2‑3 % (≈ 20‑30 kg ha⁻¹ well‑decomposed FYM or compost).  
- **Drainage:** Must not be water‑logged; use raised beds if soil is heavy clay.  
- **Salinity:** EC < 2 dS m⁻¹; leach if higher.

**Quick action plan**

1. **Soil test** (0‑15 cm, 5 points/ha) at your nearest KVK or state lab.  
2. **Adjust pH** – apply 2‑3 t ha⁻¹ lime if pH < 6.0; 1‑2 t ha⁻¹ elemental sulfur if pH > 6.8.  
3. **Add organic matter** – spread 10‑15 t ha⁻¹ FYM/compost 2‑3 weeks before transplanting.  
4. **Form raised beds** (30 cm high, 1 m wide) to improve drainage, especially on clayey fields.  
5. **Apply basal fertilizer** as per test results; a common starter dose is 60 kg N, 30 kg P₂O₅, 30 kg K₂O per hectare.  
6. **Mulch** with straw or black plastic to conserve moisture and suppre